# JSON-E
Transforms JSON into JSON (with JSON) (and Python, Go, or JS)

In [2]:
from pathlib import Path
from wxyz.jsone.widget_jsone import JSONE, W
from wxyz.lab.widget_editor import Editor
from wxyz.core.widget_json import JSON
from wxyz.lab.widget_dock import DockBox
from wxyz.yaml.widget_yaml import YAML
from yaml import safe_dump, safe_load

In [3]:
THEME = "material"

In [4]:
template_src = Editor(description="Template", mode="yaml-e", theme=THEME)
context_src = Editor(description="Context", mode="yaml", theme=THEME)
output_src = Editor(description="Output", mode="yaml", theme=THEME)

In [5]:
template = YAML()
context = YAML()
jsone = JSONE()

In [6]:
box = DockBox([
    template_src,
    context_src,
    output_src,
], layout=dict(height="40vh"))

In [7]:
W.link((template_src, "value"), (template, "source"))
W.link((context_src, "value"), (context, "source"))
W.link((context, "value"), (jsone, "context"))
W.link((template, "value"), (jsone, "source"))
W.dlink((jsone, "value"), (output_src, "value"), lambda x: safe_dump(x));

In [8]:
box

DockBox(children=(Editor(value='', description='Template', mode='yaml-e', theme='material'), Editor(value='', …

In [9]:
template_src.value, context_src.value = (
"""message: hello ${key}
k=${num}: true
""",
"""key: world
num: 1
""")

In [10]:
template_src.value, context_src.value = (
"""$mergeDeep:
- task:
    payload:
      command: [a, b]
- task:
    extra:
      foo: bar
- task:
    payload:
      command: [c]
""",
"""{}
""")

In [11]:
template_src.value, context_src.value = (
"""key: 
  $if: cond
  then: 2
other: 3
""",
"""cond: false
""")

# 😈 Using a Notebook in (@)Context
Linked Data can predictably augment a schema-constrained JSON document. While the resulting document _won't_ conform to the schema, it is suitable as an intermediate format before other activities (e.g. JSON-LD Expansion or SHACL validation) without _requiring_ a reasoner.

- Adds an `@type` to each cell (based on its `cell_type`)
  - this could be done more elgantly with `@context: {"cell_type": "nbformat:definitions/cell_type"}`
- hoists cell `metadata`
  - probably very dangerous: due to ordering, you couldn't _replace_ existing values, but with `$mergeDeep` could prepend to, say, `source` or outputs

In [12]:
try: 
    nb = __file__
except: 
    nb = "JSON-E.ipynb"

context_src.value = safe_dump({"nb": safe_load(Path(nb).read_text())})
template_src.value = """
$mergeDeep:
  - "@context":
    - gh: https://github.com/
    - nbformat: gh:jupyter/nbformat/blob/master/nbformat/v4/nbformat.v4.schema.json#
  - "@type": nbformat
  - metadata: 
      $eval: nb.metadata
  - nbformat: 
      $eval: nb.nbformat
  - nbformat_minor:
      $eval: nb.nbformat_minor
  - cells:
      $map: 
        $eval: nb.cells
      each(cell):
        $merge:
          - $eval: cell.metadata
          - "@type": nbformat:definitions/${cell.cell_type}_cell
          - $eval: cell
"""